* 1. 데이터 불러오기 & 전처리

In [1]:
import pandas as pd
label = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_label.csv")
label.head(5)

,acc_id,survival_time,amount_spent,survive
0,27835,64,0.002559,1
1,12351,64,0.120154,1
2,125437,55,0.182593,0
3,104483,64,0.016241,1
4,4704,20,0.226396,0


> 분석에 사용할 데이터프레임 만들기

In [2]:
df = label
df = df.drop(['survival_time', 'amount_spent'], axis=1)
df = df.sort_values('acc_id')
df

,acc_id,survive
22410,2,1
12651,5,0
5494,8,1
6811,17,1
16854,20,1
...,...,...
32964,130463,1
5898,130468,0
14563,130469,1
21601,130470,1


> 거래금액은 payment 데이터 활용

In [3]:
# 유저 일별 결제 금액
payment = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_payment.csv")
payment.head(5)

,day,acc_id,amount_spent
0,18,27835,0.826123
1,23,27835,0.011735
2,27,27835,0.035204
3,17,12351,0.528062
4,11,125437,0.633674


In [4]:
# 유저 아이디 기준으로 결제금액 총합 구하기
payment1 = payment.groupby('acc_id')['amount_spent'].sum().reset_index()

In [5]:
payment1

,acc_id,amount_spent
0,8,1.404644
1,20,0.896531
2,21,2.695461
3,38,6.309749
4,41,0.035204
...,...,...
23721,130447,3.215308
23722,130449,0.348521
23723,130469,0.360255
23724,130470,1.392909


> 판매유저 기준으로 거래횟수 구하기

In [6]:
# 캐릭터 간 일별 거래 이력
trade = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_trade.csv")
trade.head(5)

,day,time,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price
0,7,21:13:05,1,ag,11439,385109,48152,34247,enchant_scroll,4.793968e-08,NaN
1,2,15:33:00,1,al,22220,259705,43228,379826,enchant_scroll,4.314571e-06,NaN
2,1,14:24:53,1,al,18212,164230,38145,34209,accessory,7.190952e-08,NaN
3,7,23:13:43,1,au,121622,49839,71226,281334,adena,9.587936e-03,NaN
4,2,12:43:33,0,bd,14421,344503,65220,210558,etc,1.198492e-07,0.000639


In [7]:
# 유저 아이디 기준으로 거래횟수(사고/팔고) 구하기
trade_sell = trade.groupby('source_acc_id')['type'].count().reset_index()
trade_buy = trade.groupby('target_acc_id')['type'].count().reset_index()

In [8]:
trade_sell
# label 데이터 갯수보다 회원이 많음. 뭐지??? 

,source_acc_id,type
0,2,35
1,6,20
2,8,7
3,13,2
4,17,33
...,...,...
60640,130469,205
60641,130470,11
60642,130471,4
60643,130473,14


In [9]:
trade_sell = trade_sell.rename(columns={'source_acc_id':'acc_id', 'type':'sell_cnt'})
trade_buy = trade_buy.rename(columns={'target_acc_id':'acc_id', 'type':'buy_cnt'})

In [10]:
# 전체 데이터 합치기
df = df.merge(payment1, how='left', on='acc_id')
df = df.merge(trade_sell, how='left', on='acc_id')
df = df.merge(trade_buy, how='left', on='acc_id')
df

,acc_id,survive,amount_spent,sell_cnt,buy_cnt
0,2,1,NaN,35.0,1.0
1,5,0,NaN,NaN,10.0
2,8,1,1.404644,7.0,10.0
3,17,1,NaN,33.0,57.0
4,20,1,0.896531,9.0,45.0
...,...,...,...,...,...
39995,130463,1,NaN,1.0,NaN
39996,130468,0,NaN,NaN,2.0
39997,130469,1,0.360255,205.0,105.0
39998,130470,1,1.392909,11.0,65.0


> 일별 데이터를 유저별 합산 혹은 평균으로 변경 ___여기부터해야함

In [11]:
# --> 일단 위에 데이터들을 합으로 해서 구했으니까 여기도 합산으로 구해보기.

In [12]:
# 캐릭터 일일 활동
activity = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_activity.csv")
activity.head(5)

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,397380,aa,1.441844,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,1.10514,0.034006,0.0
1,1,75001,216231,aa,0.283219,2.247978,0.047085,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,-0.013963,0.0
2,1,75711,308769,aa,1.036910,2.956638,0.321821,0.166593,0.003470,1,0.245883,0.247337,0.0,0.0,0.00000,-0.001987,0.0
3,1,72230,387177,aa,0.229384,4.042102,0.099147,0.000000,0.001735,0,0.000000,0.000000,0.0,0.0,0.00000,-0.028884,0.0
4,1,34253,339862,aa,1.088405,0.596802,0.003377,0.000000,0.000000,0,0.000000,0.000000,0.0,0.0,0.00000,0.011777,0.0


In [13]:
activity.columns

Index(['day', 'acc_id', 'char_id', 'server', 'playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count'],
      dtype='object')

In [14]:
# 유저 아이디 기준으로 합 구하기
activity1 = activity.groupby('acc_id')[['playtime', 'npc_kill',
       'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive',
       'exp_recovery', 'fishing', 'private_shop', 'game_money_change',
       'enchant_count']].sum().reset_index()

In [15]:
activity1

,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,2,91.533663,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,-0.008746,0.0
1,5,6.116132,3.236452,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.145146,0.0
2,8,18.797802,31.172609,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,0.020353,0.0
3,17,28.464708,30.456852,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,-0.027696,0.0
4,20,25.588043,34.100184,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,-0.370192,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,130463,49.654660,0.518062,4.912516,224.327367,0.276691,0,4.425902,3.710053,0.0,0.000000,0.000000,0.010090,0.0
39996,130468,1.404393,0.200398,0.573432,0.000000,5.679234,0,0.000000,0.000000,0.0,0.965559,0.000000,-0.000091,0.0
39997,130469,57.570756,17.500566,0.206178,0.000002,0.029401,11,1.229417,1.236684,0.0,44.948304,37.120022,-0.226753,0.0
39998,130470,23.055454,25.229256,4.964914,0.232202,0.117344,4,0.737650,0.742011,0.0,0.163963,0.000000,-0.009907,0.0


In [16]:
label1 = label
label1 = label1.drop(['survival_time', 'amount_spent'], axis=1)
label1 = label1.sort_values('acc_id')
#label1 = 맨 처음 df 만든것과 동일
activity2 = activity1.merge(label1, how='left', on='acc_id')
activity2

,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,survive
0,2,91.533663,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,-0.008746,0.0,1
1,5,6.116132,3.236452,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.145146,0.0,0
2,8,18.797802,31.172609,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,0.020353,0.0,1
3,17,28.464708,30.456852,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,-0.027696,0.0,1
4,20,25.588043,34.100184,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,-0.370192,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,130463,49.654660,0.518062,4.912516,224.327367,0.276691,0,4.425902,3.710053,0.0,0.000000,0.000000,0.010090,0.0,1
39996,130468,1.404393,0.200398,0.573432,0.000000,5.679234,0,0.000000,0.000000,0.0,0.965559,0.000000,-0.000091,0.0,0
39997,130469,57.570756,17.500566,0.206178,0.000002,0.029401,11,1.229417,1.236684,0.0,44.948304,37.120022,-0.226753,0.0,1
39998,130470,23.055454,25.229256,4.964914,0.232202,0.117344,4,0.737650,0.742011,0.0,0.163963,0.000000,-0.009907,0.0,1


In [17]:
# 개별파일로 저장
# activity2.to_csv('c:/vscode/미니프로젝트/2_분류_프로젝트/data/activity_sum.csv', index=False)

> combat

In [18]:
# 캐릭터 일일 전투 활동
combat = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_combat.csv")
combat.head(5)

,day,acc_id,char_id,server,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,1,13809,54861,ap,2,13,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1,13809,256332,ap,2,13,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,1,13809,307293,ap,3,14,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,1,13809,374964,ap,7,16,0.0,0.0,0.0,0.0,0.0,0.112612,0.098129
4,1,13809,117917,ap,2,11,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [19]:
combat.columns

Index(['day', 'acc_id', 'char_id', 'server', 'class', 'level', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent'],
      dtype='object')

In [20]:
# 유저 아이디 기준으로 합 구하기
combat1 = combat.groupby('acc_id')[['level', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent']].sum().reset_index()

In [21]:
# 생존 데이터 합치기
combat2 = combat1.merge(label1, how='left', on='acc_id')
combat2

,acc_id,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,survive
0,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,5,173,0.000000,0.000000,0.000000,0.479728,0.000000,0.000000,0.098129,0
2,8,1922,0.128368,4.270875,1.183064,3.837822,0.000000,0.900892,3.041991,1
3,17,687,0.000000,0.000000,2.366129,3.358094,12.398273,0.675669,1.373803,1
4,20,624,1.283685,0.000000,1.183064,0.479728,0.000000,16.441283,10.107262,1
...,...,...,...,...,...,...,...,...,...,...
39995,130463,584,0.256737,0.000000,10.647580,3.358094,0.000000,7.770195,5.298953,1
39996,130468,43,0.000000,0.000000,0.000000,0.000000,0.000000,0.112612,0.098129,0
39997,130469,953,0.000000,0.000000,2.366129,2.878367,0.000000,0.225223,0.785030,1
39998,130470,595,0.770211,0.000000,3.549193,0.000000,0.000000,38.738365,18.350077,1


In [22]:
# 개별 파일로 저장
# combat2.to_csv('c:/vscode/미니프로젝트/2_분류_프로젝트/data/combat_sum.csv', index=False)

> pledge

In [23]:
# 캐릭터 소속 혈맹 구성원들의 일일 전투 정보
pledge = pd.read_csv("c:/vscode/미니프로젝트/2_분류_프로젝트/data/train_pledge.csv")
pledge.head(5)

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time
0,1,106660,345122,aa,381,0.036089,0.036089,0.0,0.0,0.0,0.0,0.000000,0.007078,0.027568,0.000000
1,1,46383,161773,aa,10892,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.006139
2,1,18495,321373,aa,13705,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000347
3,1,118293,366707,aa,2982,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001042
4,1,50421,158063,aa,27104,0.216536,0.072179,0.0,0.0,0.0,0.0,0.098883,0.056626,0.083978,0.000000


In [24]:
pledge.columns

Index(['day', 'acc_id', 'char_id', 'server', 'pledge_id', 'play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time'],
      dtype='object')

In [25]:
# 유저 아이디 기준으로 합 구하기
pledge1 = pledge.groupby('acc_id')[['play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt',
       'random_defender_cnt', 'same_pledge_cnt', 'temp_cnt', 'etc_cnt',
       'combat_play_time', 'non_combat_play_time']].sum().reset_index()

In [26]:
# 생존 데이터 합치기
pledge2 = pledge1.merge(label1, how='left', on='acc_id')
pledge2

,acc_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time,survive
0,5,5.557755,1.587930,0.000000,1.936569,0.929907,0.000000,2.570959,0.814001,5.097608,0.004402,0
1,8,201.306223,58.031626,2.717332,33.631748,21.745527,29.441536,92.455650,61.354404,136.879854,0.033823,1
2,17,71.312495,21.689681,0.486279,9.230979,10.872764,1.591434,36.784494,12.514373,65.693573,0.000000,1
3,20,54.891855,25.515149,7.964404,11.974452,6.580883,11.140041,28.379435,44.409036,66.451919,0.000000,1
4,21,1.660109,0.360893,0.000000,0.000000,0.071531,0.000000,0.791064,0.035391,0.501896,0.875103,1
...,...,...,...,...,...,...,...,...,...,...,...,...
33849,130463,47.710080,16.925891,0.216731,0.000000,18.025897,0.000000,28.972733,7.361396,62.492698,9.215981,1
33850,130468,0.360893,0.144357,0.000000,0.000000,0.000000,0.000000,0.197766,0.063704,0.184056,0.000000,0
33851,130469,0.902233,0.144357,0.000000,0.000000,0.000000,0.000000,0.296649,0.014157,0.229230,0.238149,1
33852,130470,18.477732,5.702112,0.165735,0.000000,3.934224,1.591434,9.492773,4.190333,7.783614,0.077838,1


In [27]:
# 개별 파일로 저장
# pledge2.to_csv('c:/vscode/미니프로젝트/2_분류_프로젝트/data/pledge_sum.csv', index=False)

> 전체 데이터 하나로 합치기

In [28]:
#activity1에서 일일아데나변동량 컬럼 삭제 (결제금액합으로만 넣을 예정)
activity11 = activity1.drop('game_money_change', axis=1)
activity11

,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,enchant_count
0,2,91.533663,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,0.0
1,5,6.116132,3.236452,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.0
2,8,18.797802,31.172609,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,0.0
3,17,28.464708,30.456852,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,0.0
4,20,25.588043,34.100184,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,130463,49.654660,0.518062,4.912516,224.327367,0.276691,0,4.425902,3.710053,0.0,0.000000,0.000000,0.0
39996,130468,1.404393,0.200398,0.573432,0.000000,5.679234,0,0.000000,0.000000,0.0,0.965559,0.000000,0.0
39997,130469,57.570756,17.500566,0.206178,0.000002,0.029401,11,1.229417,1.236684,0.0,44.948304,37.120022,0.0
39998,130470,23.055454,25.229256,4.964914,0.232202,0.117344,4,0.737650,0.742011,0.0,0.163963,0.000000,0.0


In [29]:
# 전체 데이터 합치기
df = df.merge(activity11, how='left', on='acc_id')
df = df.merge(combat1, how='left', on='acc_id')
df = df.merge(pledge1, how='left', on='acc_id')
df

,acc_id,survive,amount_spent,sell_cnt,buy_cnt,playtime,npc_kill,solo_exp,party_exp,quest_exp,...,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_y,random_defender_cnt_y,same_pledge_cnt_y,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time
0,2,1,NaN,35.0,1.0,91.533663,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,0,NaN,NaN,10.0,6.116132,3.236452,2.155434,0.695285,0.111055,...,5.557755,1.587930,0.000000,1.936569,0.929907,0.000000,2.570959,0.814001,5.097608,0.004402
2,8,1,1.404644,7.0,10.0,18.797802,31.172609,31.489136,0.000000,1.561055,...,201.306223,58.031626,2.717332,33.631748,21.745527,29.441536,92.455650,61.354404,136.879854,0.033823
3,17,1,NaN,33.0,57.0,28.464708,30.456852,9.255848,8.823094,2.322365,...,71.312495,21.689681,0.486279,9.230979,10.872764,1.591434,36.784494,12.514373,65.693573,0.000000
4,20,1,0.896531,9.0,45.0,25.588043,34.100184,0.274902,0.080060,0.083483,...,54.891855,25.515149,7.964404,11.974452,6.580883,11.140041,28.379435,44.409036,66.451919,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,130463,1,NaN,1.0,NaN,49.654660,0.518062,4.912516,224.327367,0.276691,...,47.710080,16.925891,0.216731,0.000000,18.025897,0.000000,28.972733,7.361396,62.492698,9.215981
39996,130468,0,NaN,NaN,2.0,1.404393,0.200398,0.573432,0.000000,5.679234,...,0.360893,0.144357,0.000000,0.000000,0.000000,0.000000,0.197766,0.063704,0.184056,0.000000
39997,130469,1,0.360255,205.0,105.0,57.570756,17.500566,0.206178,0.000002,0.029401,...,0.902233,0.144357,0.000000,0.000000,0.000000,0.000000,0.296649,0.014157,0.229230,0.238149
39998,130470,1,1.392909,11.0,65.0,23.055454,25.229256,4.964914,0.232202,0.117344,...,18.477732,5.702112,0.165735,0.000000,3.934224,1.591434,9.492773,4.190333,7.783614,0.077838


In [30]:
# nan값 0으로 처리
df.fillna(0, inplace=True)
df

,acc_id,survive,amount_spent,sell_cnt,buy_cnt,playtime,npc_kill,solo_exp,party_exp,quest_exp,...,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_y,random_defender_cnt_y,same_pledge_cnt_y,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time
0,2,1,0.000000,35.0,1.0,91.533663,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5,0,0.000000,0.0,10.0,6.116132,3.236452,2.155434,0.695285,0.111055,...,5.557755,1.587930,0.000000,1.936569,0.929907,0.000000,2.570959,0.814001,5.097608,0.004402
2,8,1,1.404644,7.0,10.0,18.797802,31.172609,31.489136,0.000000,1.561055,...,201.306223,58.031626,2.717332,33.631748,21.745527,29.441536,92.455650,61.354404,136.879854,0.033823
3,17,1,0.000000,33.0,57.0,28.464708,30.456852,9.255848,8.823094,2.322365,...,71.312495,21.689681,0.486279,9.230979,10.872764,1.591434,36.784494,12.514373,65.693573,0.000000
4,20,1,0.896531,9.0,45.0,25.588043,34.100184,0.274902,0.080060,0.083483,...,54.891855,25.515149,7.964404,11.974452,6.580883,11.140041,28.379435,44.409036,66.451919,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,130463,1,0.000000,1.0,0.0,49.654660,0.518062,4.912516,224.327367,0.276691,...,47.710080,16.925891,0.216731,0.000000,18.025897,0.000000,28.972733,7.361396,62.492698,9.215981
39996,130468,0,0.000000,0.0,2.0,1.404393,0.200398,0.573432,0.000000,5.679234,...,0.360893,0.144357,0.000000,0.000000,0.000000,0.000000,0.197766,0.063704,0.184056,0.000000
39997,130469,1,0.360255,205.0,105.0,57.570756,17.500566,0.206178,0.000002,0.029401,...,0.902233,0.144357,0.000000,0.000000,0.000000,0.000000,0.296649,0.014157,0.229230,0.238149
39998,130470,1,1.392909,11.0,65.0,23.055454,25.229256,4.964914,0.232202,0.117344,...,18.477732,5.702112,0.165735,0.000000,3.934224,1.591434,9.492773,4.190333,7.783614,0.077838


In [32]:
df.to_csv('c:/vscode//미니프로젝트/2_분류_프로젝트/data/total_train.csv', index=False)